In [ ]:
# %%writefile process_images_ui.py
#!/usr/bin/env python3
import os
import h5py
import numpy as np
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings('ignore', message='invalid value encountered in subtract', category=RuntimeWarning)

# ------------------------------------------------------------------------
# Custom modules (adjust or remove if not needed):
from image_processing import process_images_chunked

# ------------------------------------------------------------------------
# UI: Process Images
image_file_chooser = FileChooser(os.getcwd())
image_file_chooser.title = "Select H5 Image File"
image_file_chooser.filter_pattern = "*.h5"

mask_file_chooser = FileChooser(os.getcwd())
mask_file_chooser.title = "Select Mask H5 File"
mask_file_chooser.filter_pattern = "*.h5"

use_mask_checkbox = widgets.Checkbox(value=True, description="Use Mask")

xatol_widget = widgets.FloatText(value=0.01, description="xatol:", layout=widgets.Layout(width="140px"))
frame_interval_widget = widgets.IntText(value=10, description="Interval:", layout=widgets.Layout(width="140px"))
verbose_checkbox = widgets.Checkbox(value=False, description="Verbose")

xmin_widget = widgets.IntText(value=462, description="xmin:", layout=widgets.Layout(width="140px"))
xmax_widget = widgets.IntText(value=562, description="xmax:", layout=widgets.Layout(width="140px"))
ymin_widget = widgets.IntText(value=462, description="ymin:", layout=widgets.Layout(width="140px"))
ymax_widget = widgets.IntText(value=562, description="ymax:", layout=widgets.Layout(width="140px"))

process_images_button = widgets.Button(description="Process Images", button_style="primary")
output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '5px'})

def on_process_images_clicked(b):
    with output_area:
        clear_output()
        image_file = image_file_chooser.selected
        if not image_file:
            print("Please select an H5 image file.")
            return

        mask_file = mask_file_chooser.selected
        if not mask_file:
            print("Please select a mask H5 file.")
            return

        try:
            with h5py.File(mask_file, 'r') as f_mask:
                if use_mask_checkbox.value:
                    mask = f_mask['/mask'][:].astype(bool)
                else:
                    # If not using a mask, create an array of all True values.
                    sample = f_mask['/mask'][0]
                    mask = np.ones_like(sample, dtype=bool)
        except Exception as e:
            print("Error loading mask file:", e)
            return

        xatol_val = xatol_widget.value
        frame_interval_val = frame_interval_widget.value
        verbose_val = verbose_checkbox.value
        
        xmin_val = xmin_widget.value
        xmax_val = xmax_widget.value
        ymin_val = ymin_widget.value
        ymax_val = ymax_widget.value

        # print("Processing images to create a CSV with one row per frame...")
        process_images_chunked(
            image_file=image_file,
            mask=mask,
            frame_interval=frame_interval_val,
            xatol=xatol_val,
            fatol=10,
            n_wedges=4,
            n_rad_bins=100,
            xmin=xmin_val,
            xmax=xmax_val,
            ymin=ymin_val,
            ymax=ymax_val,
            verbose=verbose_val
        )
        print("Done.")

process_images_button.on_click(on_process_images_clicked)

process_images_ui = widgets.VBox([
    widgets.HTML("<h2>Section 1: Process Images </h2>"),
    image_file_chooser,
    mask_file_chooser,
    use_mask_checkbox,
    widgets.HBox([xatol_widget, frame_interval_widget, verbose_checkbox]),
    widgets.HBox([xmin_widget, xmax_widget, ymin_widget, ymax_widget]),
    process_images_button,
    output_area
])

def get_ui():
    """
    Returns the Process Images UI as a widget.
    """
    return process_images_ui

if __name__ == '__main__':
    ui = get_ui()
    display(ui)
